### Variabilitätsscore berechnen

In [2]:
# Daten einlesen

wetterdaten <- readRDS("datasets/rds/wetterdaten.rds")

In [3]:
# Verschiedene Informationen zu den Dateien anzeigen
# durch entfernen der Kommentarzeichen können die einzelnen Informationen ausgegeben werden

for (i in seq_along(wetterdaten)) {
  #print(names(wetterdaten)[i])
  #print(paste("Anzahl der Zeilen", names(wetterdaten[i]), ": ", nrow(wetterdaten[[i]])))
  #print(summary(wetterdaten[[i]]))
  #print(head(wetterdaten[[1]]))
  #print(nrow(wetterdaten[[1]]))
  #boxplot(wetterdaten[[i]]$TMK, main = names(wetterdaten)[i])
  #print(subset(wetterdaten[[i]], RSK > 400.0 | RSK < 0.0))
  #print(subset(wetterdaten[[i]], SDK > 24 | SDK < 0))
  #print(subset(wetterdaten[[i]], TMK > 60 | TMK < -60))
  #plot(wetterdaten[[i]]$MESS_DATUM, wetterdaten[[i]]$TMK, type = "l", col = "blue", xlab = "Datum", ylab = "Temperatur", main = names(wetterdaten)[i])
  #boxplot(wetterdaten[[i]]$TMK, main = names(wetterdaten)[i])
  #break
}

In [4]:
# Matrix für Standardabweichung (6 Spalten, n Zeilen)
n <- length(wetterdaten)
ergebnis_matrix <- matrix(numeric(n * 6), nrow = n, ncol = 6)

# Benenne die Spalten der Matrix
colnames(ergebnis_matrix) <- c("Varianz_RSK", "Stda_RSK", 
                               "Varianz_SDK", "Stda_SDK", 
                               "Varianz_TMK", "Stda_TMK")

for (i in seq_along(wetterdaten)) {
  ergebnis_matrix[i, 1] <- var(wetterdaten[[i]]$RSK, na.rm = TRUE)
  ergebnis_matrix[i, 2] <- sqrt(ergebnis_matrix[i, 1])
  
  ergebnis_matrix[i, 3] <- var(wetterdaten[[i]]$SDK, na.rm = TRUE)
  ergebnis_matrix[i, 4] <- sqrt(ergebnis_matrix[i, 3])
  
  ergebnis_matrix[i, 5] <- var(wetterdaten[[i]]$TMK, na.rm = TRUE)
  ergebnis_matrix[i, 6] <- sqrt(ergebnis_matrix[i, 5])
}
print(ergebnis_matrix)


      Varianz_RSK Stda_RSK Varianz_SDK Stda_SDK Varianz_TMK Stda_TMK
 [1,]    39.06270 6.250016    21.71427 4.659858    54.41073 7.376363
 [2,]    19.18963 4.380597    19.73699 4.442633    37.61751 6.133311
 [3,]    79.90281 8.938837    20.05801 4.478617    50.34378 7.095335
 [4,]    50.11535 7.079220    21.28061 4.613091    55.95097 7.480038
 [5,]    31.45721 5.608673    22.05088 4.695837    54.35087 7.372305
 [6,]    16.42201 4.052408    20.77976 4.558483    33.75741 5.810113
 [7,]    77.30649 8.792411    16.39719 4.049344    56.84773 7.539743
 [8,]    16.99618 4.122642    21.23359 4.607992    56.47594 7.515048
 [9,]    14.38721 3.793048    23.10893 4.807175    42.68266 6.533197
[10,]    14.38272 3.792455    19.78939 4.448527    48.66625 6.976120


In [5]:
ergebnis_df <- as.data.frame(ergebnis_matrix)
print(ergebnis_df)

   Varianz_RSK Stda_RSK Varianz_SDK Stda_SDK Varianz_TMK Stda_TMK
1     39.06270 6.250016    21.71427 4.659858    54.41073 7.376363
2     19.18963 4.380597    19.73699 4.442633    37.61751 6.133311
3     79.90281 8.938837    20.05801 4.478617    50.34378 7.095335
4     50.11535 7.079220    21.28061 4.613091    55.95097 7.480038
5     31.45721 5.608673    22.05088 4.695837    54.35087 7.372305
6     16.42201 4.052408    20.77976 4.558483    33.75741 5.810113
7     77.30649 8.792411    16.39719 4.049344    56.84773 7.539743
8     16.99618 4.122642    21.23359 4.607992    56.47594 7.515048
9     14.38721 3.793048    23.10893 4.807175    42.68266 6.533197
10    14.38272 3.792455    19.78939 4.448527    48.66625 6.976120
